In [3]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.simplefilter('ignore')

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [4]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
%%time

df = pd.read_parquet('df_clean_withtokens.parquet', engine='pyarrow')
df.shape

CPU times: user 23 s, sys: 6.23 s, total: 29.3 s
Wall time: 21.2 s


(170984, 11)

### Topic Modelling

In [7]:
df.head()

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,clean_text_tokens
0,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,2021,3,18,3,Artificial intelligence improves parking effic...,Artificial intelligence improves parking effic...,"[artificial, intelligence, improves, parking, ...","[artificial, intelligence, improves, parking, ..."
1,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,2020,2,27,3,Children With Autism Saw Their Learning and So...,Children With Autism Saw Their Learning and So...,"[children, autism, saw, learning, social, skil...","[children, autism, saw, learning, social, skil..."
2,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...",2021,3,26,4,"Forget ML, AI and Industry 4.0 obsolescence sh...","Forget ML, AI and Industry 4.0 obsolescence sh...","[forget, ml, ai, industry, obsolescence, focus...","[forget, ml, ai, industry, obsolescence, focus..."
3,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,2021,3,10,2,Strategy Analytics: 71 of Smartphones Sold Glo...,Strategy Analytics: 71 of Smartphones Sold Glo...,"[strategy, analytics, smartphones, sold, globa...","[strategy, analytics, smartphones, sold, globa..."
4,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,2020,10,20,1,Olympus to Support Endoscopic AI Diagnosis Edu...,Olympus to Support Endoscopic AI Diagnosis Edu...,"[olympus, support, endoscopic, ai, diagnosis, ...","[olympus, support, endoscopic, ai, diagnosis, ..."


In [8]:
df = df.drop(["text", "title"], axis=1)

In [12]:
# df["clean_title_tokens"] = df["clean_title_tokens"].str.replace('\x00', '').parallel_apply(eval)
# df["clean_text_tokens"] = df["clean_text_tokens"].parallel_apply(eval)

In [13]:
# Creating a term dictionary and document term matrix
dictionary_n_title = corpora.Dictionary(df["clean_title_tokens"])
dictionary_n_text = corpora.Dictionary(df["clean_text_tokens"])

In [14]:
doc_term_matrix_n_title = [dictionary_n_title.doc2bow(doc) for doc in df["clean_title_tokens"]]
doc_term_matrix_n_text = [dictionary_n_text.doc2bow(doc) for doc in df["clean_text_tokens"]]

In [15]:
# Creating a Latent Dirichlet Allocation(LDA) Model for topic modelling
Lda = gensim.models.ldamodel.LdaModel

In [16]:
import multiprocessing
num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors

print(f'Using {workers} workers')

Using 64 workers


In [17]:
def initial_5_topic_mult(doc_term_matrix,dictionary):
    lda_model = LdaMulticore(corpus=doc_term_matrix,
                   id2word=dictionary,
                   num_topics=10,
                   random_state=100,
                   passes=10,
                   eta='auto',
                   workers=workers)

    for i in lda_model.print_topics(num_words=7):
        print(i)
    doc_lda = lda_model[doc_term_matrix]

In [18]:
%time initial_10_topic_mult(doc_term_matrix_n_title, dictionary_n_title)

(0, '0.059*"ai" + 0.019*"news" + 0.019*"chatgpt" + 0.014*"google" + 0.013*"bard" + 0.010*"global" + 0.007*"data"')
(1, '0.090*"ai" + 0.013*"news" + 0.010*"new" + 0.009*"artificial" + 0.009*"intelligence" + 0.006*"microsoft" + 0.005*"business"')
(2, '0.071*"ai" + 0.018*"data" + 0.015*"news" + 0.012*"chatgpt" + 0.009*"new" + 0.008*"platform" + 0.007*"science"')
(3, '0.060*"intelligence" + 0.059*"artificial" + 0.046*"ai" + 0.009*"tech" + 0.008*"using" + 0.007*"chatgpt" + 0.007*"news"')
(4, '0.058*"ai" + 0.021*"data" + 0.016*"science" + 0.012*"first" + 0.011*"technology" + 0.011*"chatgpt" + 0.010*"platform"')
(5, '0.062*"market" + 0.039*"artificial" + 0.039*"intelligence" + 0.033*"ai" + 0.022*"global" + 0.017*"growth" + 0.017*"analysis"')
(6, '0.096*"ai" + 0.009*"news" + 0.007*"intelligence" + 0.006*"artificial" + 0.006*"health" + 0.005*"new" + 0.005*"help"')
(7, '0.084*"ai" + 0.020*"chatgpt" + 0.016*"new" + 0.010*"news" + 0.006*"technology" + 0.005*"microsoft" + 0.005*"intelligence"')
(8,

In [19]:
%time initial_10_topic_mult(doc_term_matrix_n_text, dictionary_n_text)

(0, '0.010*"ai" + 0.008*"machine" + 0.008*"learning" + 0.008*"republic" + 0.006*"new" + 0.006*"market" + 0.005*"news"')
(1, '0.010*"ai" + 0.007*"news" + 0.007*"market" + 0.006*"industry" + 0.006*"public" + 0.005*"business" + 0.005*"technology"')
(2, '0.013*"data" + 0.012*"ai" + 0.007*"business" + 0.006*"customer" + 0.006*"news" + 0.005*"new" + 0.005*"company"')
(3, '0.011*"ai" + 0.007*"company" + 0.006*"stock" + 0.005*"ha" + 0.004*"new" + 0.004*"market" + 0.004*"news"')
(4, '0.012*"ago" + 0.008*"hour" + 0.007*"ai" + 0.006*"new" + 0.005*"news" + 0.005*"said" + 0.005*"day"')
(5, '0.008*"ai" + 0.006*"new" + 0.005*"intelligence" + 0.005*"ha" + 0.005*"artificial" + 0.004*"news" + 0.004*"data"')
(6, '0.012*"ai" + 0.007*"news" + 0.006*"data" + 0.005*"paid" + 0.005*"technology" + 0.004*"business" + 0.004*"program"')
(7, '0.013*"ai" + 0.007*"gray" + 0.006*"group" + 0.006*"technology" + 0.006*"media" + 0.005*"health" + 0.005*"patient"')
(8, '0.037*"market" + 0.017*"ai" + 0.012*"intelligence" + 0